In [8]:
import requests
import pandas as pd
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
from db_connection import start_engine
from datetime import date

engine = start_engine()

In [9]:
LAT = 53.5753
LON = 10.0153
years = range(2022,2026)

API_URL = "https://air-quality-api.open-meteo.com/v1/air-quality"

In [10]:
def air_pollution(year, name):
    today = date.today()
    if year == today.year:
        end_date = today.strftime("%Y-%m-%d")
    else:
        end_date = f"{year}-12-31"
    params = {
        "latitude": LAT,
        "longitude": LON,
        "hourly": "pm10,pm2_5,nitrogen_dioxide,ozone,carbon_monoxide,sulphur_dioxide",
        "start_date": f"{year}-01-01",
        "end_date": end_date,
    }

    data = requests.get(API_URL, params=params).json()

    df = pd.DataFrame({
        "timestamp": data["hourly"]["time"],
        "pm10": data["hourly"]["pm10"],
        "pm2_5": data["hourly"]["pm2_5"],
        "no2": data["hourly"]["nitrogen_dioxide"],
        "o3": data["hourly"]["ozone"],
        "co": data["hourly"]["carbon_monoxide"],
        "so2": data["hourly"]["sulphur_dioxide"],
    })

    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.to_sql(name = name, con = engine, # Push to DB
                           if_exists= "replace", index = False)



In [11]:
for n in years:
    print(n)
    air_pollution(n, f"air_quality_{n}")

2022
2023
2024
2025
